# FinTech Bootcamp Project 2

### Evaluating The Risk of Making Investments in India

##### This project will evaluate the risk in investing in India by comparing the results of similar investments that offer a fixed rate of interest. The reason for this evaluation is that investments with a fixed rate of interest in India offer a higher rate of interest than the same time of investment in the United States. However, it is possible that changes in the exchange rate between the Indian Rupee and the US Dollar could negate the potential benefits of making an investment in India. This is why this project will compare a ten year, 100,000 dollar investment in the United States with the same investment in India, and predict the most likely exchange rate after this ten year period. This will be done by using the previous ten years of exchange rate data and creating a model to predict the rate after another ten years based off of the previous rates.

## Steps for the Project

#### 1. Simulate a 10 year 100,000 dollar investment in both the United States given the current average Certificate Of Deposit (CD) rates
#### 2. Convert 100,000 dollars to rupees based off of the current exchange rate and simulate a 10 year investment given the average Fixed Deposit (FD) rates in India
#### 3. Access the previous 10 years of exchange rate datas and use it to fit a Facebook Prophet Model
#### 4. Use this Prophet Model to predict the most probable exchange rate after 10 years
#### 5. Convert the Indian investment back in to US Dollars given the predicted future exchange rate and compare the returns of both investments

In [9]:
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
from forex_python.converter import CurrencyRates
import mplfinance as mpf

from pathlib import Path
from prophet import Prophet
import hvplot.pandas
%matplotlib inline

from easy_exchange_rates import API

### Step 1

In [37]:
us_invest = 100000
ind_invest = 100000

us_rate = 0.5
ind_rate = 6

time_period = 10

In [38]:
date = datetime.datetime(2021, 9, 1)

c = CurrencyRates()
curr_rate = c.get_rate('USD', 'INR', date)
display(curr_rate)
ind_invest = ind_invest * curr_rate
display(ind_invest)

73.07099940763307

7307099.940763307

In [39]:
def compound_interest(principle, rate, time):
 
    Amount = principle * (pow((1 + rate / 100), time))
    CI = Amount - principle
    return (CI)

In [40]:
us_compounded = us_invest + compound_interest(us_invest, us_rate, time_period)
ind_compounded = ind_invest + compound_interest(ind_invest, ind_rate, time_period)

In [41]:
display(us_compounded)
display(ind_compounded)

105114.01320407896

13085903.097324397

### Step 2

In [42]:
api = API()

In [43]:
time_series = api.get_exchange_rates(
  base_currency="USD", 
  start_date="2020-09-01", 
  end_date="2021-09-01", 
  targets=["INR"]
)

data_frame = api.to_dataframe(time_series)

In [44]:
time_series = api.get_exchange_rates(
  base_currency="USD", 
  start_date="2019-09-01", 
  end_date="2020-09-01", 
  targets=["INR"]
)

df_2 = api.to_dataframe(time_series)

time_series = api.get_exchange_rates(
  base_currency="USD", 
  start_date="2018-09-01", 
  end_date="2019-09-01", 
  targets=["INR"]
)

df_3 = api.to_dataframe(time_series)

time_series = api.get_exchange_rates(
  base_currency="USD", 
  start_date="2017-09-01", 
  end_date="2018-09-01", 
  targets=["INR"]
)

df_4 = api.to_dataframe(time_series)

time_series = api.get_exchange_rates(
  base_currency="USD", 
  start_date="2016-09-01", 
  end_date="2017-09-01", 
  targets=["INR"]
)

df_5 = api.to_dataframe(time_series)

time_series = api.get_exchange_rates(
  base_currency="USD", 
  start_date="2015-09-01", 
  end_date="2016-09-01", 
  targets=["INR"]
)

df_6 = api.to_dataframe(time_series)

time_series = api.get_exchange_rates(
  base_currency="USD", 
  start_date="2014-09-01", 
  end_date="2015-09-01", 
  targets=["INR"]
)

df_7 = api.to_dataframe(time_series)

time_series = api.get_exchange_rates(
  base_currency="USD", 
  start_date="2013-09-01", 
  end_date="2014-09-01", 
  targets=["INR"]
)

df_8 = api.to_dataframe(time_series)

time_series = api.get_exchange_rates(
  base_currency="USD", 
  start_date="2012-09-01", 
  end_date="2013-09-01", 
  targets=["INR"]
)

df_9 = api.to_dataframe(time_series)

time_series = api.get_exchange_rates(
  base_currency="USD", 
  start_date="2011-09-01", 
  end_date="2012-09-01", 
  targets=["INR"]
)

df = api.to_dataframe(time_series)

In [45]:
df_2 = df_2.append(data_frame)
df_3 = df_3.append(df_2)
df_4 = df_4.append(df_3)
df_5 = df_5.append(df_4)
df_6 = df_6.append(df_5)
df_7 = df_7.append(df_6)
df_8 = df_8.append(df_7)
df_9 = df_9.append(df_8)
df = df.append(df_9)

display(df.head())
display(df.tail())

,INR
2011-09-01,45.749565
2011-09-02,45.715888
2011-09-03,45.715888
2011-09-04,45.715888
2011-09-05,45.827801


,INR
2021-08-28,73.405916
2021-08-29,73.428846
2021-08-30,73.343519
2021-08-31,72.957325
2021-09-01,72.973936


### Step 3

In [46]:
df['date'] = df.index
df.columns = ['y', 'ds']
display(df.shape)
display(df.head())
display(df.tail())

(3660, 2)

,y,ds
2011-09-01,45.749565,2011-09-01
2011-09-02,45.715888,2011-09-02
2011-09-03,45.715888,2011-09-03
2011-09-04,45.715888,2011-09-04
2011-09-05,45.827801,2011-09-05


,y,ds
2021-08-28,73.405916,2021-08-28
2021-08-29,73.428846,2021-08-29
2021-08-30,73.343519,2021-08-30
2021-08-31,72.957325,2021-08-31
2021-09-01,72.973936,2021-09-01


In [47]:
model = Prophet()
model

In [48]:
model.fit(df)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [51]:
future_trends = model.make_future_dataframe(periods=120, freq='M')
display(future_trends.head())
display(future_trends.tail())

,ds
0,2011-09-01
1,2011-09-02
2,2011-09-03
3,2011-09-04
4,2011-09-05


,ds
3769,2031-04-30
3770,2031-05-31
3771,2031-06-30
3772,2031-07-31
3773,2031-08-31


In [52]:
forecast_trends = model.predict(future_trends)
forecast_trends.head()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2011-09-01,47.680993,46.799151,49.520246,47.680993,47.680993,0.462406,0.462406,0.462406,0.030782,0.030782,0.030782,0.431625,0.431625,0.431625,0.0,0.0,0.0,48.143399
1,2011-09-02,47.706581,46.776228,49.609258,47.706581,47.706581,0.439866,0.439866,0.439866,0.006638,0.006638,0.006638,0.433227,0.433227,0.433227,0.0,0.0,0.0,48.146447
2,2011-09-03,47.732169,46.777616,49.449001,47.732169,47.732169,0.433262,0.433262,0.433262,0.001561,0.001561,0.001561,0.431701,0.431701,0.431701,0.0,0.0,0.0,48.165431
3,2011-09-04,47.757758,46.782788,49.517863,47.757758,47.757758,0.431053,0.431053,0.431053,0.003942,0.003942,0.003942,0.427111,0.427111,0.427111,0.0,0.0,0.0,48.188810
4,2011-09-05,47.783346,46.728461,49.410194,47.783346,47.783346,0.321930,0.321930,0.321930,-0.097638,-0.097638,-0.097638,0.419568,0.419568,0.419568,0.0,0.0,0.0,48.105276


In [53]:
forecast_trends = forecast_trends.set_index(['ds'])
forecast_trends.tail()

,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
ds,,,,,,,,,,,,,,,,,,
2031-04-30,89.566760,-193.334325,344.387334,-194.595708,343.909622,-0.043827,-0.043827,-0.043827,0.030351,0.030351,0.030351,-0.074178,-0.074178,-0.074178,0.0,0.0,0.0,89.522933
2031-05-31,89.696193,-199.519665,348.852606,-199.339369,347.689346,-0.055701,-0.055701,-0.055701,0.001561,0.001561,0.001561,-0.057262,-0.057262,-0.057262,0.0,0.0,0.0,89.640492
2031-06-30,89.821452,-206.153822,352.589454,-203.975353,351.441376,0.228758,0.228758,0.228758,-0.097638,-0.097638,-0.097638,0.326396,0.326396,0.326396,0.0,0.0,0.0,90.050210
2031-07-31,89.950885,-208.074618,355.476187,-208.513028,355.466178,-0.283334,-0.283334,-0.283334,0.030782,0.030782,0.030782,-0.314116,-0.314116,-0.314116,0.0,0.0,0.0,89.667551
2031-08-31,90.080319,-211.717827,360.565724,-212.358419,359.447774,0.430817,0.430817,0.430817,0.003942,0.003942,0.003942,0.426875,0.426875,0.426875,0.0,0.0,0.0,90.511136


#### The yhat, or most likely outcome, for the end of the term is 90.51 rupees to the dollar 

### Step 4

In [57]:
new_exchange = 90.51
ind_compounded = ind_compounded / new_exchange
ind_compounded

144579.63868439285

In [58]:
us_compounded

105114.01320407896

In [60]:

print (ind_compounded / 100000)
print (us_compounded / 100000)

1.4457963868439285
1.0511401320407896


#### After ten years, the investment in India is likely to end up returning 144,579.63 dollars and the investment in the United States is likely to end up returning 105,114.01 dollars, which is about a 37.5% greater return